In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np
import pandas as pd 
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
from sklearn.preprocessing import scale 
from imblearn.over_sampling import SMOTE

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score,roc_curve
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn import tree

from sklearn.multiclass import OneVsRestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC

from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import AdaBoostRegressor
from catboost import CatBoostRegressor

from sklearn import metrics
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import r2_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy.stats import shapiro
from sklearn import preprocessing

from warnings import filterwarnings
filterwarnings('ignore')

from sklearn import metrics
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv("/kaggle/input/health-insurance-cross-sell-prediction/train.csv")
test = pd.read_csv("/kaggle/input/health-insurance-cross-sell-prediction/test.csv")

In [ ]:
train.drop("id", axis = 1, inplace = True)

In [ ]:
test.drop("id", axis = 1, inplace = True)

In [ ]:
train

In [ ]:
def Missing_Values(data):
    variable_name=[]
    total_value=[]
    total_missing_value=[]
    missing_value_rate=[]
    unique_value_list=[]
    total_unique_value=[]
    data_type=[]
    for col in data.columns:
        variable_name.append(col)
        data_type.append(data[col].dtype)
        total_value.append(data[col].shape[0])
        total_missing_value.append(data[col].isnull().sum())
        missing_value_rate.append(round(data[col].isnull().sum()/data[col].shape[0],3))
        unique_value_list.append(data[col].unique())
        total_unique_value.append(len(data[col].unique()))
    missing_data=pd.DataFrame({"Variable":variable_name,"Total_Value":total_value,\
                             "Total_Missing_Value":total_missing_value,"Missing_Value_Rate":missing_value_rate,
                             "Data_Type":data_type,"Unique_Value":unique_value_list,\
                               "Total_Unique_Value":total_unique_value})
    return missing_data.sort_values("Missing_Value_Rate",ascending=False)

In [ ]:
Missing_Values(train)

In [ ]:
train["Gender"] = train["Gender"].replace("Male", 0)
train["Gender"] = train["Gender"].replace("Female", 1)



train["Vehicle_Damage"] = train["Vehicle_Damage"].replace("No", 0)
train["Vehicle_Damage"] = train["Vehicle_Damage"].replace("Yes", 1)





In [ ]:
# Male = 0
# Female = 1

# Vehicle_Damage Yes = 1
# Vehicle_Damage No = 0
train

In [ ]:
train.corr()

In [ ]:
train.shape

In [ ]:
train_dummies = pd.get_dummies(train)

In [ ]:
train_dummies

In [ ]:
train_dummies.groupby("Response").count()

In [ ]:
ratio = 46710 / 334399
ratio

In [ ]:
train_dummies.rename(columns={"Vehicle_Age_1-2 Year": "Vehicle_Age_1_2 Year", 
                              "Vehicle_Age_< 1 Year": "Vehicle_Age_0_1_Year", "Vehicle_Age_> 2 Years":"Vehicle_Age_2_inf"},inplace = True)

In [ ]:
X = train_dummies.drop("Response", axis = 1)
y = train_dummies["Response"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [ ]:
def plot_confusion_matrix(y_real, y_pred):
    cm = confusion_matrix(y_real, y_pred)

    ax= plt.subplot()
    sns.heatmap(cm, annot=True, ax = ax, fmt='g')

    ax.set_xlabel('Predicted labels')
    ax.set_ylabel('True labels')

In [ ]:
X_train

# XGB

The important thing is to catch customers who are not interested so estimating 0s is more important for us.


In [ ]:
# 1 : Customer is interested, 0 : Customer is not interested
XGB = XGBClassifier(scale_pos_weight=1)
XGB_model = XGB.fit(X_train, y_train)
y_pred = XGB_model.predict(X_test)

xgb_roc_auc_score = roc_auc_score(y_test, y_pred)
ac = accuracy_score(y_test, y_pred)
class_reportt = classification_report(y_test, y_pred)


print("AC: {}\nROC AUC: {}\nCLASS REPORT:\n {}".format(ac,xgb_roc_auc_score,class_reportt))

In [ ]:
y_probs = XGB_model.predict_proba(X_test)[:,1] # This will give you positive class prediction probabilities  
y_pred = np.where(y_probs > 0.5, 1, 0)

xgb_roc_auc_score = roc_auc_score(y_test, y_pred)
ac = accuracy_score(y_test, y_pred)
class_reportt = classification_report(y_test, y_pred)

print("AC: {}\nROC AUC: {}\nCLASS REPORT:\n {}".format(ac,xgb_roc_auc_score,class_reportt))


Let's check the Importance of values 

In [ ]:
Importance=pd.DataFrame({"Importance":XGB_model.feature_importances_*100},
                       index=X_train.columns)
Importance.sort_values(by="Importance",
                      axis=0,ascending=True).plot(kind="barh",color="r")
plt.xlabel("Feature Importance")

ımportance_sort_values = Importance.sort_values("Importance", ascending = False)
#ımportance_sort_values[ımportance_sort_values['Importance']>1]
ımportance_sort_values

I fit the model with the best parameters:

In [ ]:
XGB = XGBClassifier(scale_pos_weight=3, learning_rate = 0.1, n_estimators = 100,  max_depth = 5,  min_child_weight= 7, colsample_bytree = 0.5)
XGB_model = XGB.fit(X_train, y_train)
y_pred = XGB_model.predict(X_test)

xgb_roc_auc_score = roc_auc_score(y_test, y_pred)
ac = accuracy_score(y_test, y_pred)
class_reportt = classification_report(y_test, y_pred)


print("AC: {}\nROC AUC: {}\nCLASS REPORT:\n {}".format(ac,xgb_roc_auc_score,class_reportt))

In [ ]:
y_probs = XGB_model.predict_proba(X_test)[:,1] # This will give you positive class prediction probabilities  
y_pred = np.where(y_probs > 0.5, 1, 0)

xgb_roc_auc_score = roc_auc_score(y_test, y_pred)
ac = accuracy_score(y_test, y_pred)
class_reportt = classification_report(y_test, y_pred)

print("AC: {}\nROC AUC: {}\nCLASS REPORT:\n {}".format(ac,xgb_roc_auc_score,class_reportt))

In [ ]:
Importance=pd.DataFrame({"Importance":XGB_model.feature_importances_*100},
                       index=X_train.columns)
Importance.sort_values(by="Importance",
                      axis=0,ascending=True).plot(kind="barh",color="r")
plt.xlabel("Feature Importance")

ımportance_sort_values = Importance.sort_values("Importance", ascending = False)
#ımportance_sort_values[ımportance_sort_values['Importance']>1]
ımportance_sort_values

In [ ]:
def plot_roc_curve(fpr, tpr,model_name):
    plt.figure(figsize=(5,5))
    plt.title(model_name)
    plt.plot(fpr,tpr, label = roc_auc_score) 
    plt.plot([0,1],ls='--')
    plt.plot([0,0],[1,0],c='.5')
    plt.plot([1,1],c='.5')
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()

In [ ]:
fpr, tpr, threshold = metrics.roc_curve(y_test, y_pred)
plot_roc_curve(fpr, tpr, "XGB  ROC CURVE")

As a result  roc curve is 0.76 

f1 score is 0.47 for 1. I think it could be better.


This is my first kernel. What do you think about importance and Roc Curve(did not satisfy me) waiting for your comment.